In [1]:
# !pip install timm -q
# 如果需要 albumentations 最新版
# !pip install -U albumentations -q

import os
# 安装 timm
os.system('pip install timm -q')
print("✅ Environment setup complete.")
import os
import sys
import numpy as np
import pandas as pd
import random
import torch

✅ Environment setup complete.


In [ ]:
# === 全局配置 ===
class Config:
    # 基础参数
    SEED = 42
    IMG_SIZE = 224
    BATCH_SIZE = 32     # 如果显存炸了，改成 16
    EPOCHS = 5          # Kaggle上跑5轮做演示
    LR = 3e-4
    NUM_WORKERS = 2     # Kaggle 环境设太大容易 Shared Memory Error
    
    # 设备
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    
    # 路径占位符
    # DATA_ROOT_DIR = ""
    # TRAIN_CSV = ""
    # TRAIN_IMG_DIR = ""

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    print(f"✅ Random Seed Set: {seed}")

# === 数据集自动发现与用户指定逻辑 ===
def setup_directories():
    print("🔍 Searching for dataset...")
    
    # 常见的 Kaggle 数据集挂载路径
    candidate_paths = [
        "/kaggle/input/siim-isic-melanoma-classification",
        "../input/siim-isic-melanoma-classification"
    ]
    
    found_path = None
    for p in candidate_paths:
        if os.path.exists(p) and os.path.exists(os.path.join(p, "train.csv")):
            found_path = p
            break
    
    # 如果找不到，强制等待用户输入
    if found_path is None:
        print("\n❌ Warning: Could not automatically find 'train.csv'.")
        print("Please look at the 'Data' panel on the right side.")
        user_input = input("👉 Please PASTE the full path to the dataset directory (where train.csv is located): ")
        user_input = user_input.strip()
        
        if os.path.exists(os.path.join(user_input, "train.csv")):
            found_path = user_input
        else:
            raise FileNotFoundError(f"Could not find train.csv in {user_input}")

    # 路径确认
    Config.DATA_ROOT_DIR = found_path
    Config.TRAIN_CSV = os.path.join(found_path, "train.csv")
    
    # 检查 jpeg 目录 (有时候是 train/, 有时候是 jpeg/train/)
    possible_img_dirs = ["jpeg/train", "train"]
    img_dir_final = None
    for sub in possible_img_dirs:
        check = os.path.join(found_path, sub)
        if os.path.exists(check):
            img_dir_final = check
            break
            
    if img_dir_final is None:
         # 再次fallback，让用户手动指定图片位置
         img_dir_final = input(f"👉 found train.csv but NOT image dir. Paste full path to train images folder: ")
    
    Config.TRAIN_IMG_DIR = img_dir_final
    
    print("\n✅ Dataset Configured Successfully:")
    print(f"   CSV Path: {Config.TRAIN_CSV}")
    print(f"   Img Path: {Config.TRAIN_IMG_DIR}")

# 运行配置
seed_everything(Config.SEED)
setup_directories()

In [ ]:
import cv2
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2

class MelanomaDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        
        self.image_ids = self.df['image_name'].values
        self.targets = self.df['target'].values
        
        # === 核心逻辑：定义有效特征与因果黑名单 ===
        # 1. 剔除 ID 类信息
        # 2. 剔除 数据泄露列 (diagnosis, benign_malignant)
        # 3. 剔除 因果权重列
        ignore_cols = [
            'image_name', 'patient_id', 'target', 'causal_weight', 
            'diagnosis', 'benign_malignant' 
        ]
        
        # 筛选剩下的列作为 tabular features
        self.meta_cols = [c for c in self.df.columns if c not in ignore_cols]
        self.meta_features = self.df[self.meta_cols].values
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # 1. 读图
        img_id = self.image_ids[idx]
        img_path = os.path.join(self.img_dir, f"{img_id}.jpg")
        image = cv2.imread(img_path)
        
        # 防御性代码
        if image is None:
            image = np.zeros((Config.IMG_SIZE, Config.IMG_SIZE, 3), dtype=np.uint8)
        else:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # 2. 增强
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        
        # 3. 提取特征和标签
        meta = torch.tensor(self.meta_features[idx], dtype=torch.float32)
        target = torch.tensor(self.targets[idx], dtype=torch.float32)
        
        # 4. 【因果推断接口】提取权重
        if 'causal_weight' in self.df.columns:
            weight = torch.tensor(self.df.iloc[idx]['causal_weight'], dtype=torch.float32)
        else:
            weight = torch.tensor(1.0, dtype=torch.float32)
            
        return image, meta, target, weight

def get_preprocessed_df(csv_path):
    print("⏳ Preprocessing CSV data...")
    df = pd.read_csv(csv_path)
    
    # 填充缺失值
    # 注意：真实项目中可能要先根据 Training Set 算 mean，然后应用到 Test Set
    df['anatom_site_general_challenge'] = df['anatom_site_general_challenge'].fillna('unknown')
    df['sex'] = df['sex'].fillna('unknown')
    
    # 处理年龄 (填均值 + 简单归一化)
    df['age_approx'] = df['age_approx'].fillna(df['age_approx'].mean())
    df['age_approx'] = df['age_approx'] / 100.0
    
    # One-Hot 编码 (转换文本为数字)
    df = pd.get_dummies(df, columns=['sex', 'anatom_site_general_challenge'], prefix=['sex', 'site'], dtype=float)
    
    print(f"✅ Feature processing done. Total samples: {len(df)}")
    return df

import torch.nn as nn
import timm

class CausalFusionModel(nn.Module):
    def __init__(self, meta_features_dim, out_dim=1, pretrained=True):
        super().__init__()
        
        # 1. Image Branch
        # Kaggle 无网络环境下如果timmn download失败，可能需要手动上传权重
        # 正常有网状态下设为 True 即可
        self.backbone = timm.create_model('efficientnet_b0', pretrained=pretrained, num_classes=0, in_chans=3)
        self.img_dim = self.backbone.num_features # 1280
        
        # 2. Tabular Branch
        self.meta_net = nn.Sequential(
            nn.Linear(meta_features_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.ReLU()
        )
        self.meta_dim = 128
        
        # 3. Heads
        total_dim = self.img_dim + self.meta_dim
        
        # Main Prediction Head
        self.classifier = nn.Linear(total_dim, out_dim)
        
        # 【因果接口】 Auxiliary Head for De-confounding
        self.confounder_head = nn.Linear(total_dim, 5) # Placeholder

    def forward(self, image, meta, return_features=False):
        # Image -> [B, 1280]
        x_img = self.backbone(image)
        # Table -> [B, 128]
        x_meta = self.meta_net(meta)
        
        # Concatenate -> [B, 1408]
        # 这是因果表征的核心层
        features = torch.cat((x_img, x_meta), dim=1)
        
        logits = self.classifier(features)
        
        if return_features:
            return logits, features
        
        return logits

In [ ]:
import torch.optim as optim
from sklearn.metrics import roc_auc_score
from torch.utils.data import random_split
from tqdm.notebook import tqdm # 使用 notebook 专用的进度条

def train_epoch(model, loader, criterion, optimizer, epoch):
    model.train()
    running_loss = 0.0
    # 在 Notebook 里尽量减少 tqdm 打印频率
    
    for images, metas, targets, weights in tqdm(loader, desc=f"Epoch {epoch} Train", leave=False):
        images, metas = images.to(Config.DEVICE), metas.to(Config.DEVICE)
        targets = targets.to(Config.DEVICE).unsqueeze(1)
        weights = weights.to(Config.DEVICE).unsqueeze(1)
        
        optimizer.zero_grad()
        
        # 前向
        logits = model(images, metas)
        
        # 因果 Loss 计算
        loss_per_sample = criterion(logits, targets)
        loss = (loss_per_sample * weights).mean()
        
        # 反向
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    return running_loss / len(loader)

def validate_epoch(model, loader, criterion):
    model.eval()
    val_loss = 0.0
    preds = []
    truths = []
    
    with torch.no_grad():
        for images, metas, targets, _ in tqdm(loader, desc="Validating", leave=False):
            images, metas = images.to(Config.DEVICE), metas.to(Config.DEVICE)
            targets_gpu = targets.to(Config.DEVICE).unsqueeze(1)
            
            logits = model(images, metas)
            loss = criterion(logits, targets_gpu)
            val_loss += loss.item()
            
            # 记录结果算 AUC
            probs = torch.sigmoid(logits).cpu().numpy()
            preds.extend(probs)
            truths.extend(targets.numpy())
            
    try:
        auc = roc_auc_score(truths, preds)
    except:
        auc = 0.5
        
    return val_loss / len(loader), auc

# === 主执行流 ===
def main():
    # 1. 准备数据
    df = get_preprocessed_df(Config.TRAIN_CSV)
    
    # 图像增强 (Training)
    train_transforms = A.Compose([
        A.Resize(Config.IMG_SIZE, Config.IMG_SIZE),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15, p=0.5), # 增加一点难度
        A.Normalize(),
        ToTensorV2()
    ])
    
    # 图像增强 (Validation - 只做 Normalize)
    val_transforms = A.Compose([
        A.Resize(Config.IMG_SIZE, Config.IMG_SIZE),
        A.Normalize(),
        ToTensorV2()
    ])
    
    # 为了演示简单，我们直接 split 这个 DataFrame
    # 实际 Kaggle 比赛应该使用 StratifiedKFold
    from sklearn.model_selection import train_test_split
    df_train, df_val = train_test_split(df, test_size=0.2, random_state=Config.SEED, stratify=df['target'])
    
    # 构建 Datasets
    train_ds = MelanomaDataset(df_train.reset_index(drop=True), Config.TRAIN_IMG_DIR, transform=train_transforms)
    val_ds = MelanomaDataset(df_val.reset_index(drop=True), Config.TRAIN_IMG_DIR, transform=val_transforms)
    
    # 构建 DataLoaders
    train_loader = DataLoader(train_ds, batch_size=Config.BATCH_SIZE, shuffle=True, 
                              num_workers=Config.NUM_WORKERS, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=Config.BATCH_SIZE, shuffle=False, 
                            num_workers=Config.NUM_WORKERS, pin_memory=True)
    
    print(f"📊 Training Samples: {len(train_ds)} | Validation Samples: {len(val_ds)}")
    
    # 2. 初始化模型
    # 获取动态的表格列数
    meta_dim = train_ds.meta_features.shape[1]
    print(f"🧠 Meta Features Dim: {meta_dim}")
    
    model = CausalFusionModel(meta_features_dim=meta_dim).to(Config.DEVICE)
    
    # 3. 设置训练组件
    # 因果加权训练要求 Loss Reduction 为 None
    # pos_weight=torch.tensor([20.0]) 用于应对类别极其不平衡 (因为皮肤癌阳性样本很少)
    # 在这个 MVP 中我们可以先注释掉 pos_weight 看原生效果，或者手动设置
    criterion = nn.BCEWithLogitsLoss(reduction='none') 
    optimizer = optim.AdamW(model.parameters(), lr=Config.LR)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=Config.EPOCHS)
    
    # 4. 开始 Loop
    best_auc = 0.0
    for epoch in range(1, Config.EPOCHS + 1):
        print(f"\n--- Epoch {epoch}/{Config.EPOCHS} ---")
        
        train_loss = train_epoch(model, train_loader, criterion, optimizer, epoch)
        val_loss, val_auc = validate_epoch(model, val_loader, criterion)
        
        scheduler.step()
        
        print(f"Train Loss: {train_loss:.4f}")
        print(f"Valid Loss: {val_loss:.4f} | AUC: {val_auc:.4f}")
        
        if val_auc > best_auc:
            best_auc = val_auc
            torch.save(model.state_dict(), "best_causal_model.pth")
            print("🌟 Best Model Saved!")
            
    print(f"\n✅ Training Complete. Best AUC: {best_auc:.4f}")

if __name__ == "__main__":
    main()